# Using the EBAY API 

## Finding API 2022


<img align="left" width="100" height="100" src="ebay.png">

## Install EBAY SDK and required packages

In [ ]:
import pandas as pd
import numpy as np
from ebaysdk.exception import ConnectionError
from ebaysdk.finding import Connection as Finding
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv("ebay_api_key.env")

In [ ]:
print(os.getcwd())
!python3 --version

In [ ]:
# Deprecation
# pd.set_option('max_colwidth', 1000)
# pd.set_option('max_rows', 1000)
pd.set_option("display.max_rows", 500)

## Hide your Application ID in a .env file

In [ ]:
APP_ID = os.getenv("EBAY_PROD_ID")

In [ ]:
APP_ID

In [ ]:
def get_results(payload):
    try:
        api = Finding(appid= APP_ID, config_file=None)
        response = api.execute("findItemsAdvanced", payload)
        return response.dict()
    except ConnectionError as e:
        print(e)
        print(e.response.dict())

In [11]:
payload = {"keywords":"knife", 
          "categoryId":"48818",
          "itemFilter.name":"Condition",
          "itemFilter.value":"Used",
          "paginationInput.entriesPerPage":5,
          "paginationInput.pageNumber":15
          }

In [12]:
payload

{'keywords': 'knife',
 'categoryId': '48818',
 'itemFilter.name': 'Condition',
 'itemFilter.value': 'Used',
 'paginationInput.entriesPerPage': 5,
 'paginationInput.pageNumber': 15}

<findItemsAdvancedRequest xmlns="http://www.ebay.com/marketplace/search/v1/services">
  <itemFilter>
    <name>Seller</name>
      <!--Enter an eBay UserID that exist in the environment 
        (Sandbox, Production, etc.) you are using -->
    <value>xx</value>
  </itemFilter>
  <paginationInput>
    <entriesPerPage>3</entriesPerPage>
  </paginationInput>
  <outputSelector>SellerInfo</outputSelector>
</findItemsAdvancedRequest>


In [ ]:
results = get_results(payload)

## Function to get the payload

### Define the payload

In [1]:
import requests
import json

url = "https://ebay-sold-items-api.herokuapp.com/findCompletedItems"

payload = json.dumps({
  "keywords": "knife",
  "max_search_results": "240",
  "category_id": "48818",
  "remove_outliers": True,
  "site_id": "0",
  "aspects": [
    {
      "name": "Condition",
      "value": "Used"
    },
  ]
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)


In [2]:
def get_total_pages(results):
    '''Get the total number of pages from the results'''
    if results:
        return int(results.get('paginationOutput').get('totalPages'))
    else:
        return

In [ ]:
get_total_pages(results)

In [13]:
r = response.json()

In [14]:
r

{'success': True,
 'average_price': 36.84,
 'median_price': 20.25,
 'min_price': 0.5,
 'max_price': 215.5,
 'results': 228,
 'response_url': 'https://www.ebay.com/sch/48818/i.html?_nkw=knife&LH_Sold=1&LH_Complete=1&_ipg=240&Condition=Used',
 'products': [{'title': 'Souvenir Folding Knives One With Wolves One With Train In Case VGC',
   'sale_price': 20,
   'date_sold': 'Jun 14, 2022',
   'link': 'https://www.ebay.com/itm/165214235944?hash=item2677892d28:g:rusAAOSw9TNhqYVQ'},
  {'title': 'Victorinox Climber Swiss Army Knife Silvertech',
   'sale_price': 21,
   'date_sold': 'Jun 14, 2022',
   'link': 'https://www.ebay.com/itm/144591687337?hash=item21aa560ea9:g:fwAAAOSwDN9in4Mv'},
  {'title': '#839 Opinel INOX No. 08 Folding Knife France Mirror Finished Blade ',
   'sale_price': 21.5,
   'date_sold': 'Jun 14, 2022',
   'link': 'https://www.ebay.com/itm/374122427999?hash=item571b6f365f:g:tScAAOSwavtiolGI'},
  {'title': 'A.G. Russell Single Blade Folding Pocket Knife RUS-AG715Y IN Original 

In [ ]:
def search_ebay(payload):
    '''parse the response - results and concatentate to the dataframe'''
    results = get_results(payload)
    total_pages = get_total_pages(results)
    items_list = results['searchResult']['item']
        
    i = 2
    while(i <= total_pages):
        payload['paginationInput'] = {'entriesPerPage': 100, 'pageNumber': i}        
        results = get_results(payload)
        items_list.extend(results['searchResult']['item'])
        i += 1
        
    df_items = pd.DataFrame(columns=['itemId', 'title', 'viewItemURL', 'galleryURL', 'location', 'postalCode',
                                 'paymentMethod''listingType', 'bestOfferEnabled', 'buyItNowAvailable',
                                 'currentPrice', 'bidCount', 'sellingState'])

    for item in items_list:
        row = {
            'itemId': item.get('itemId'),
            'title': item.get('title'),
            'viewItemURL': item.get('viewItemURL'),
            'galleryURL': item.get('galleryURL'),
            'location': item.get('location'),
            'postalCode': item.get('postalCode'),
            'paymentMethod': item.get('paymentMethod'),        
            'listingType': item.get('listingInfo').get('listingType'),
            'bestOfferEnabled': item.get('listingInfo').get('bestOfferEnabled'),
            'buyItNowAvailable': item.get('listingInfo').get('buyItNowAvailable'),
            'currentPrice': item.get('sellingStatus').get('currentPrice').get('value'),
            'bidCount': item.get('bidCount'),
            'sellingState': item.get('sellingState'),
        }

        ## Deprecation note - don't use append
        '''https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html'''
        
        new_df = pd.DataFrame([row])
        df_items = pd.concat([df_items, new_df],axis=0, ignore_index=True)


    return df_items


# Run the search

In [ ]:
df3 = search_ebay(payload)

In [ ]:
df3.to_csv('CRKT_Spyd_Kershaw.csv', index=False)

In [ ]:
df3.to_csv('Case_Camillus_Gerber_Emerson.csv', index=False)

In [ ]:
df4 = search_ebay(payload)

In [ ]:
df4.to_csv('SOG_Shrade.csv', index=False)

In [ ]:
df.to_csv('temp_save.csv', index=False)

In [ ]:
df.head()